This notebook imports a table of VAA attributes of all Lower 48 network stream segments in the NHDPlusV2.1.  We subset these data into a set of attributes that we know we need for network operations and pickle these data for fast read access for future steps.  We also pickle all attributes for other efforts where we may need additional attributes.

In [17]:
#Read in source data and build binary format for quick reading of data
import pandas as pd
import requests
import io
import urllib.request as ur
import numpy as np

In [3]:
#Download data from ScienceBase source: https://www.sciencebase.gov/catalog/item/5d126fa3e4b0941bde56ea73
download_url = 'https://www.sciencebase.gov/catalog/file/get/5d126fa3e4b0941bde56ea73?f=__disk__c9%2Fad%2F57%2Fc9ad57469af8f37fc1a5577cc75a080cf39ea8b2'


download_file = 'flow_table.zip'
ur.urlretrieve(download_url, download_file)

('flow_table.zip', <http.client.HTTPMessage at 0x170bd362048>)

In [33]:
#Import data into pandas from zip txt file
download_file = 'flow_table.zip'
df = pd.read_csv(download_file)

#create float version of segment ids (comids as float)
df['seg_id'] = df['COMID'].astype(float)
df['watershed_id'] =df['VPUID'].astype(str)

update_watersheds = {'10U': '10', '10L': '10', '7': '07', '5': '05', '8': '08', '03W': '03', '03N': '03', '2': '02', '4': '04', '1': '01', '6': '06', '03S': '03', '9': '09'}
df = df.replace({'watershed_id':update_watersheds})

#update nan in rpuid
df['proc_unit'] = df['RPUID']
df['proc_unit'] = np.where(df.proc_unit.isnull(), (df['watershed_id'].astype(str)+'a'), (df['proc_unit'].astype(str)))

#Update watersheds to represent drainages of US 
#05,06,07,08,10U,10L,11 = MISSISSIPPI
#01 = New England
#02 = Mid Atlantic
#03N, 03S, 03W = South Atlantic Gulf
#04 = Great Lakes
#09 = Souris Red-Rainy
#12 = Texas Gulf
#13 = Rio Grande
#14, 15 = Colorado
#16 = Great Basin
#17 = Pacific Northwest
#18 = California
update_watersheds = {'05':'11','06':'11','07':'11','08':'11','10':'11','14':'15'}
df = df.replace({'watershed_id':update_watersheds})

In [34]:
#Shows number of records per watershed 
l = df['watershed_id'].tolist()
import collections
counter=collections.Counter(l)
print(counter)

Counter({'11': 1243141, '03': 330422, '17': 232810, '15': 187008, '18': 142613, '02': 129670, '04': 107692, '16': 96269, '12': 68901, '01': 67906, '13': 55854, '09': 29053})


In [35]:
#Create subset of dataset, this subset includes data that are needed for many network operations
##################################################################################################
#Select needed fields
network_df_sub = df[['seg_id','COMID', 'FTYPE' ,'AreaSqKM', 'StreamOrde', 'TotDASqKM', 'REACHCODE', 'LENGTHKM', 'StreamOrde','StartFlag', 'FromMeas', 'ToMeas','FromNode', 'ToNode','watershed_id','proc_unit']]
#Remove Coastline flowlines
network_df_sub = network_df_sub.loc[network_df_sub['FTYPE']!='Coastline']


In [36]:
#Serialize subset of data for faster read
network_df_sub.to_pickle('flow_table_sub.pkl')

In [37]:
#serialize all of the data in case we need additional attributes
#df['seg_id'] = df['COMID'].astype(float)
df.to_pickle('flow_table_all.pkl')